In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

2024-04-23 21:21:59.695209: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-23 21:21:59.701521: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 21:21:59.787458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 21:21:59.787518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 21:21:59.789232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [3]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    print([train_x.shape[1]])
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=np.mean(train_x), variance=np.var(train_x)),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [4]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [5]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [7]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


[11]                                                 
  0%|          | 0/8 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

46/46 [==============================] - 1s 5ms/step - loss: 3.2580 - root_mean_squared_error: 1.8050 - val_loss: 1.2904 - val_root_mean_squared_error: 1.1359

Epoch 2/3                                            

46/46 [==============================] - 0s 3ms/step - loss: 0.9867 - root_mean_squared_error: 0.9933 - val_loss: 0.8114 - val_root_mean_squared_error: 0.9008

Epoch 3/3                                            

46/46 [==============================] - 0s 4ms/step - loss: 0.8037 - root_mean_squared_error: 0.8965 - val_loss: 0.7556 - val_root_mean_squared_error: 0.8693

12/12 [==============================] - 0s 2ms/step - loss: 0.7556 - root_mean_squared_error: 0.8693

  0%|          | 0/8 [00:01<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: /tmp/tmpql_z7uzk/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpql_z7uzk/model/data/model/assets


[11]                                                                           
 12%|█▎        | 1/8 [00:05<00:41,  5.98s/trial, best loss: 0.8692699074745178]

/mnt/b464db71-2681-49c1-bf62-871043a830fd/class/mlflow_class/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



...

[11]                                                                           
Epoch 1/3                                                                      

46/46 [==============================] - 1s 6ms/step - loss: 29.5004 - root_mean_squared_error: 5.4314 - val_loss: 27.7989 - val_root_mean_squared_error: 5.2725

Epoch 2/3                                                                      

46/46 [==============================] - 0s 2ms/step - loss: 25.9356 - root_mean_squared_error: 5.0927 - val_loss: 24.4965 - val_root_mean_squared_error: 4.9494

Epoch 3/3                                                                      

46/46 [==============================] - 0s 2ms/step - loss: 22.7856 - root_mean_squared_error: 4.7734 - val_loss: 21.5770 - val_root_mean_squared_error: 4.6451

12/12 [==============================] - 0s 2ms/step - loss: 21.5770 - root_mean_squared_error: 4.6451

 88%|████████▊ | 7/8 [00:41<00:05,  5.65s/trial, best loss: 0.8604480624198914]INFO:ten

INFO:tensorflow:Assets written to: /tmp/tmpgxw9opum/model/data/model/assets


100%|██████████| 8/8 [00:45<00:00,  5.71s/trial, best loss: 0.8604480624198914]
INFO:tensorflow:Assets written to: /tmp/tmprhyeoc9l/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmprhyeoc9l/model/data/model/assets


Best parameters: {'lr': 0.027715442907988143, 'momentum': 0.5305243801827795}
Best eval rmse: 0.8604480624198914
